In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from EvGym import config

In [37]:
save = True
#df_imb_pre = pd.read_csv("./data/imbalance_prices_2019.csv", parse_dates=["Date"])
df_imb_pre = pd.read_csv('./data/tennet/settlementprices_history.csv', parse_dates=["Date"])
df_imb_pre.head()

/tmp/ipykernel_35149/1889231914.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imb_pre = pd.read_csv('./data/tennet/settlementprices_history.csv', parse_dates=["Date"])


,Date,PTE,period_from,period_until,upward_incident_reserve,downward_incident_reserve,To regulate up,To regulate down,Incentive component,Consume,Feed,Regulation state
0,2014-01-01,1,00:00,00:15,NaN,NaN,NaN,7.40,0,7.40,7.40,-1
1,2014-01-01,2,00:15,00:30,NaN,NaN,NaN,14.46,0,14.46,14.46,-1
2,2014-01-01,3,00:30,00:45,NaN,NaN,NaN,15.40,0,15.40,15.40,-1
3,2014-01-01,4,00:45,01:00,NaN,NaN,NaN,-43.01,0,-43.01,-43.01,-1
4,2014-01-01,5,01:00,01:15,NaN,NaN,NaN,20.18,0,20.18,20.18,-1


In [38]:
df_imb_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210336 entries, 0 to 210335
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Date                       210336 non-null  datetime64[ns]
 1   PTE                        210336 non-null  int64         
 2   period_from                210336 non-null  object        
 3   period_until               210336 non-null  object        
 4   upward_incident_reserve    812 non-null     object        
 5   downward_incident_reserve  11 non-null      object        
 6   To regulate up             115664 non-null  float64       
 7   To regulate down           120435 non-null  float64       
 8   Incentive component        210336 non-null  int64         
 9   Consume                    210336 non-null  float64       
 10  Feed                       210336 non-null  float64       
 11  Regulation state           210336 non-null  int64   

In [39]:
df_imb_pre["datetime"] = df_imb_pre["Date"] + pd.to_timedelta(df_imb_pre["period_from"]+':00')
df_imb_pre["datehour"] = df_imb_pre["datetime"].dt.strftime("%Y-%m-%d %H:00:00")
df_imb_pre["ts_cte"] = (df_imb_pre["datetime"] - config.starttime_min).dt.total_seconds() / config.timestep
df_imb_pre = df_imb_pre.sort_values(by="datetime").reset_index(drop=True)
df_imb_pre["price_da"] = 0
df_imb_pre["feed"] = df_imb_pre["Feed"] / 1000
df_imb_pre["consume"] = df_imb_pre["Consume"] / 1000

In [40]:
df_imb_pre[["datetime", "datehour", "ts_cte", "Date", "consume", "feed"]].describe()

,datetime,ts_cte,Date,consume,feed
count,210336,210336.000000,210336,210336.000000,210336.000000
mean,2016-12-31 12:27:34.792332544,149028.459665,2016-12-31 00:35:04.792332288,0.043585,0.039840
min,2014-01-01 00:00:00,122736.000000,2014-01-01 00:00:00,-0.500000,-0.500000
25%,2015-07-02 18:56:15,135882.937500,2015-07-02 00:00:00,0.025820,0.024690
50%,2016-12-31 11:52:30,149027.875000,2016-12-31 00:00:00,0.034175,0.032710
75%,2018-07-02 06:48:45,162174.812500,2018-07-02 00:00:00,0.045890,0.043390
max,2019-12-31 23:45:00,175319.750000,2019-12-31 00:00:00,0.936120,0.936120
std,NaN,15179.741843,NaN,0.059441,0.057332


In [ ]:
def make_point(df_imb_pre, imb_col):
    df_imb = df_imb_pre[df_imb_pre["datetime"].dt.minute == 0].copy()
    df_imb = df_imb.rename(columns={"imb_col": "price_im", "datetime": "date"}).reset_index(drop=True)
    df_imb = df_imb.sort_values(by="datetime").reset_index(drop=True)
    df_imb["ts"] = df_imb.index + df_imb["ts_cte"].min()
    df_imb = df_imb[["date", "price_da", "price_im", "ts"]]
    return df_imb

In [41]:
# Make 4 prices
quant = [0, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1]
# 1. Feed when minutes == 0
df_imb_a = df_imb_pre[df_imb_pre["datetime"].dt.minute == 0].copy()
df_imb_a = df_imb_a.rename(columns={"feed": "price_im", "datetime": "date"}).reset_index(drop=True)
df_imb_a = df_imb_a[["date", "price_da", "price_im", "ts_cte"]]
df_imb_a["ts"] = df_imb_a.index + df_imb_a["ts_cte"].min()
print("a----\n", len(df_imb_a), df_imb_a["price_im"].quantile(quant))


# 2. Feed averaged over the hour
df_imb_b = df_imb_pre.groupby(by="datehour").agg(
                                                price_da = ('price_da', 'mean'),
                                                price_im = ('feed', 'mean'),
                                                ts = ('ts', 'min'),
                                                 ).reset_index()

df_imb_b = df_imb_b.rename(columns={"feed": "price_im", "datehour": "date"}).reset_index(drop=True)
df_imb_b = df_imb_b[["date", "price_da", "price_im", "ts"]]
print("b----\n", len(df_imb_b), df_imb_b["price_im"].quantile(quant))

# 3. Consume when minutes == 0
df_imb_c = df_imb_pre[df_imb_pre["datetime"].dt.minute == 0].copy()
df_imb_c = df_imb_c.rename(columns={"consume": "price_im", "datetime": "date"}).reset_index(drop=True)
df_imb_c = df_imb_c[["date", "price_da", "price_im", "ts"]]
print("c----\n", len(df_imb_c), df_imb_c["price_im"].quantile(quant))

# 4. Consume averaged over the hour
df_imb_d = df_imb_pre.groupby(by="datehour").agg(
                                                price_da = ('price_da', 'mean'),
                                                price_im = ('consume', 'mean'),
                                                ts = ('ts', 'min'),
                                                 ).reset_index()

df_imb_d = df_imb_d.rename(columns={"feed": "price_im", "datehour": "date"}).reset_index(drop=True)
df_imb_d = df_imb_d[["date", "price_da", "price_im", "ts"]]
print("d----\n", len(df_imb_d), df_imb_d["price_im"].quantile(quant))
# Columns: date (datetime), price_da (all zeros), price_im, ts
# Also a version without outliers

a----
 52584 0.00   -0.500000
0.01   -0.150097
0.10    0.005780
0.25    0.023137
0.50    0.032190
0.75    0.044860
0.90    0.097937
0.99    0.350020
1.00    0.764920
Name: price_im, dtype: float64


KeyError: "Column(s) ['ts'] do not exist"

In [7]:
def smooth(x):
    median = x.median()
    iqr = x.quantile(0.75) - x.quantile(0.25)
    y = np.tanh((x - median) / (2*iqr)) * 2*iqr + median
    return y

In [8]:
df_imb_as = df_imb_a.copy()
df_imb_as["price_im"] = smooth(df_imb_as["price_im"])
df_imb_bs = df_imb_b.copy()
df_imb_bs["price_im"] = smooth(df_imb_bs["price_im"])
df_imb_cs = df_imb_c.copy()
df_imb_cs["price_im"] = smooth(df_imb_cs["price_im"])
df_imb_ds = df_imb_d.copy()
df_imb_ds["price_im"] = smooth(df_imb_ds["price_im"])

In [9]:
# Save a, b, c, d, as, bs, cs, ds
if save:
    df_imb_a.to_csv("./data/df_prices_2019_a.csv", index=False)
    df_imb_b.to_csv("./data/df_prices_2019_b.csv", index=False)
    df_imb_c.to_csv("./data/df_prices_2019_c.csv", index=False)
    df_imb_d.to_csv("./data/df_prices_2019_d.csv", index=False)
    df_imb_as.to_csv("./data/df_prices_2019_as.csv", index=False)
    df_imb_bs.to_csv("./data/df_prices_2019_bs.csv", index=False)
    df_imb_cs.to_csv("./data/df_prices_2019_cs.csv", index=False)
    df_imb_ds.to_csv("./data/df_prices_2019_ds.csv", index=False)

In [10]:
df_imb_a[df_imb_a["ts"] == 168698]

,date,price_da,price_im,ts
2138,2019-03-31 02:00:00,0,0.3376,168698.0


In [15]:
df_prueba = pd.DataFrame([[1, 1], [1, 2], [3, 1]], columns = ["a", "b"])
df_prueba = df_prueba.sort_values(by = ["a"]).reset_index(drop=True)
df_prueba["ts"] = df_prueba.index +10
df_prueba

,a,b,ts
0,1,1,10
1,1,2,11
2,3,1,12
